In [8]:
from mobilechelonian import Turtle
import random
import time
import pickle
import pickle
import simpleaudio as sa
from keyword_spotting.feature_extraction.utils import read_wav
import numpy as np
from scipy.io.wavfile import write
import pyaudio
import wave
import tensorflow as tf
from keyword_spotting.model import models
from keyword_spotting.feature_extraction.utils import extract_features, read_wav, windowed
import numpy as np
from keyword_spotting.predictions import labels

In [6]:
colours=["red","blue","yellow","brown","black","purple","green"]

def up(t):
    new_color = random.choice(colours)
    t.pencolor(new_color)
    t.penup()
def down(t):
    new_color = random.choice(colours)
    t.pencolor(new_color)
    t.pendown()
def right(t):
    new_color = random.choice(colours)
    t.pencolor(new_color)
    t.right(90)
    t.forward(40) 
def left(t):
    new_color = random.choice(colours)
    t.pencolor(new_color)
    t.right(90)
    t.forward(40) 

def forward(t):
    t.forward(40)
    
def home(t):
    t.home()
    
commands = {
    'right': right,
    'left': left,
    'down': down,
    'up': up,
    'off': home, 
    'go': forward,
}

available_commands = ['right', 'left', 'down', 'up', 'off', 'go']

def record_command(filename):
    chunk = 1024  # Record in chunks of 1024 samples
    sample_format = pyaudio.paInt16  # 16 bits per sample
    channels = 1
    fs = 16000  # Record at 44100 samples per second
    seconds = 2

    p = pyaudio.PyAudio()  # Create an interface to PortAudio

    print('Say a command')

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []  # Initialize array to store frames

    # Store data in chunks for 3 seconds
    for i in range(0, int(fs / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)

    # Stop and close the stream 
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()

    print('Finished recording')

    # Save the recorded data as a WAV file
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()
    
def load_model():
    configFILE='/Users/emi/cnn_residual2_19_04_14_Jun_13_2021_results.pkl'
    modelFILE = '/Users/emi/cnn_residual2_19_04_14_Jun_13_2021'
    with open(configFILE, 'rb') as f:
        config, time, predictions = pickle.load(f)
    input_shape = [100, 40]
    number_of_classes=12
    model = models[config["model"]["name"]](input_shape, number_of_classes, **config["model"]["params"])
    # Restore the weights
    model.load_weights(f'{modelFILE}')
    return model
    
def predict_action(filename, model):
    sample_rate, signal = read_wav(filename)
    data = extract_features(signal, sample_rate)
    data = np.expand_dims(data[:100,:], axis=0)
    predictions = model.predict(data)
    label_predicted = np.argmax(predictions)
    return labels[label_predicted]

def play_turtle():
    t = Turtle()
    model = load_model()
    command = True
    total_commands=0
    while command:
        time.sleep(2)
        filename=f'command_{total_commands}.wav'
        record_command(filename)
        command_voice =predict_action(filename, model)
        if command_voice not in available_commands:
            continue
        print(command_voice)
        commands[command_voice](t)
        total_commands+=1
        if command_voice == 'off':
            command=False
    print('Game Finished. Thanks for play with us :D')

In [9]:
play_turtle()

Turtle()

Say a command
Finished recording
up
Say a command
Finished recording
go
Say a command
Finished recording
up
Say a command
Finished recording
up
Say a command
Finished recording
off
Game Finished. Thanks for play with us :D
